# Compsci 690V Homework 5

We pick the mini challenge 1 of VAST Challenge 2008 for this homework. 

**Description:** The Paraiso movement is controversial and is having considerable social impact in a specific area of the world. We have extracted a segment of the Paraiso (the movement) Wikipedia edits page. Please note this is not the Paraiso Manifesto Wiki page which is part of the background materials, but a related different page. Please use visual analytics to describe the social relationships of the editors (those that have edited/modified the Wikipedia page) as they are reflected in these files.



To obtain a dataset similar to the one given in the mini challenge, we scraped the edits of the wikipedia page **"2016–17 Kashmir unrest"** (link: https://en.wikipedia.org/w/index.php?title=2016%E2%80%9317_Kashmir_unrest&limit=500&action=history). We chose this dataset since it is a recent revolutionary movement simirarly to the one described in the challenge. We scraped the text using the script in the file **getEdits.py**.

We parsed the text of each commit into a dataframes with the columns:
* **timestamp** - the timestamp of the edit
* **user** - the user or ip of the editor
* **minorEdit** - True or False - if the edit is minor
* **pageLength** - Length of the page after the edit (in Bytes)
* **editDiff** - bytes changed by the edit
* **comment** - comment or description of the edit
* **tags**
* **entireEdit** - entire raw edit text

In [86]:
import numpy as np
import pandas as pd
import re

file = open('data/wiki sample data.txt','r')

lines = file.readlines()
#print (lines)

df = pd.DataFrame(columns=['timestamp','user','minorEdit','pageLength','editDiff','comment','tags','entireEdit'])
i=0
for line in lines:
        entireEdit = line
        #split via brackets
        token = re.split('\(|\)',line)
        #print(token)
        timestamp = token[2].split('?')[0]
        timestamp = pd.to_datetime(timestamp)
        #split by ?
        user = token[2].split('?')[1:]
        user = ''.join(user)
        user = user[1:-1]
        m = token[4]
        m= m.split('.')[0]
        m= True if (m=='? m ') else False
        byte = token[5].split(' ')[0]
        byte = int(byte.replace(',',''))
        change = token[7]
        change = int(change.replace(',',''))
        comment = re.split('\. \. ',line)[-1]
        tagComment = comment.split('(undo)')
        #print (tagComment)
        comment = tagComment[0][1:-2]
        #undo is not at the end
        if (len(tagComment)>1):
            tag = tagComment[1][2:-2]
            tag = tag.split(':')
            if (len(tag)>1):
                tag = tag[1][1:]
                tag = tag.split(',')
            if (tag[0]==''):
                tag =[]
        else:
            tag=''
        
        #tag = tag.split(',')
        #print (tag)
        
        df.loc[i]=[timestamp,user,m,byte,change,comment,tag,entireEdit]
        i+=1

print (df.shape)
#print (df.head)

(1338, 8)
<bound method NDFrame.head of                timestamp                                   user minorEdit  \
0    2017-10-09 03:59:00                             GreenC bot      True   
1    2017-10-03 11:05:00                              Kautilya3     False   
2    2017-10-03 11:04:00                              Kautilya3     False   
3    2017-10-03 09:05:00                          Hamzah Banday     False   
4    2017-10-03 08:59:00                          Hamzah Banday     False   
5    2017-10-03 08:57:00                          Hamzah Banday     False   
6    2017-09-19 14:41:00                             Mfarazbaig     False   
7    2017-09-19 14:32:00                       MBlaze Lightning     False   
8    2017-09-19 14:21:00                             Mfarazbaig     False   
9    2017-09-18 14:09:00                       MBlaze Lightning     False   
10   2017-09-17 12:44:00                         111.92.143.153     False   
11   2017-09-14 15:57:00            

In [87]:


POI1 = df['user'].mode().astype(str)
#print (POI1)

mask = df['user']=='DinoBambinoNFS'
POI1df = df[mask]['comment']
#print (POI1df)

commentsPOI1 = list(POI1df)
#print (commentsPOI1)

In [88]:
import nltk
import re
commentsDf = df['comment']

from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer("english")

def tokenizeAndStem(text):
    CommentsTokens=[]
    #for userComments in commentsDf:
    CommentsTokens = (nltk.word_tokenize(text))
    #filter out punctuations and numeric tokens
    filtered_tokens = []
    for token in CommentsTokens:
        if re.search('[a-zA-Z]', token):
            filtered_tokens.append(token)
    stems = [stemmer.stem(t) for t in filtered_tokens]
    return stems

totalvocab_stemmed = []
for i in commentsDf:
    stem = tokenizeAndStem(i)
    totalvocab_stemmed.extend(stem)

print (len(totalvocab_stemmed))

from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer(max_df=0.8,
                                 min_df=0, stop_words='english',
                                 use_idf=True,tokenizer=tokenizeAndStem, ngram_range=(1,3))

tfidf_matrix = tfidf_vectorizer.fit_transform(commentsDf)
print (tfidf_matrix.shape)
terms = tfidf_vectorizer.get_feature_names()
#print (terms)



7056
(1338, 5959)


In [89]:
from sklearn.cluster import KMeans

num_clusters = 5

km = KMeans(n_clusters=num_clusters)
km.fit(tfidf_matrix)

clusters = km.labels_.tolist()
#print (clusters)
df['clusters']  =clusters
#print (df.head())
print (df['clusters'].value_counts())
#print (km.cluster_centers_.argsort()[:,::-1])

from sklearn.manifold import MDS

MDS()

# convert two components as we're plotting points in a two-dimensional plane
# "precomputed" because we provide a distance matrix
# we will also specify `random_state` so the plot is reproducible.
mds = MDS(n_components=2, random_state=1)

pos = mds.fit_transform(tfidf_matrix.toarray())  # shape (n_components, n_samples)

xs, ys = pos[:, 0], pos[:, 1]


2    1198
4      78
0      48
1      11
3       3
Name: clusters, dtype: int64


In [68]:
print (xs[:5],ys[:5])

[-0.46511467 -0.51887076 -0.31686017 -0.70098599  0.00086644] [  9.33524327e-01  -7.12312556e-01  -7.84299525e-01  -4.59945760e-01
   2.78039524e-04]


In [90]:
def get_colors(clusters):
    colors=[]
    for i in clusters:
        if i==0:
            colors.append('red')
        if i==1:
            colors.append('blue')
        if i==2:
            colors.append('green')
        if i==3:
            colors.append('brown')
        if i==4:
            colors.append('yellow')
    return colors

colors = get_colors(clusters)

from bokeh.plotting import figure
from bokeh.io import output_notebook, show
from bokeh.models import ColumnDataSource, Select,LabelSet

output_notebook()
a=figure(x_range =list(set(list(df['user']))),plot_height=500, plot_width=900, title='clustering groups based on edits')
data = ColumnDataSource(data=dict(x=df['user'],y=df['timestamp'],colors = colors,label=df['user']))
a.square('x','y',fill_color='colors',line_color='colors',source=data,size=10)
labels = LabelSet(x='user', y='timestamp', text='user', level='glyph',
                  x_offset=5, y_offset=5, source=ColumnDataSource(df), render_mode='canvas')
#a.add_layout(labels)
show(a)

print(data.column_names)
b=figure( plot_height=500, plot_width=900, title='clustering groups based on edits')
data1 = ColumnDataSource(data=dict(x=xs,y=ys,colors = colors))
b.square('x','y',fill_color='colors',line_color='colors',source=data1,size=10)
labels = LabelSet(x='x', y='y', text='label', level='glyph',
                  x_offset=5, y_offset=5, source=data,render_mode='canvas')
b.add_layout(labels)
show(b)

Loading BokehJS ...

['x', 'y', 'colors', 'label']
